In [1]:
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats
import glob
import os
import re

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

from optimization_mctoptw_solomon_dataset_maut.aco import ACO
from optimization_mctoptw_solomon_dataset_maut.acs import ACS
from optimization_mctoptw_solomon_dataset_maut.bso import BSO
from optimization_mctoptw_solomon_dataset_maut.dka import DKA
from optimization_mctoptw_solomon_dataset_maut.ga import GA
from optimization_mctoptw_solomon_dataset_maut.pso import PSO
from optimization_mctoptw_solomon_dataset_maut.sa import SA
from optimization_mctoptw_solomon_dataset_maut.tabu_search import TS
from optimization_mctoptw_solomon_dataset_maut.woa import WOA
from optimization_mctoptw_solomon_dataset_maut.ils import ILS
from optimization_mctoptw_solomon_dataset_maut.ils_sa import ILS_SA
from optimization_mctoptw_solomon_dataset_maut.ils_ts import ILS_TS
from optimization_mctoptw_solomon_dataset_maut.ils_sa_ts import ILS_SA_TS
from optimization_mctoptw_solomon_dataset_maut.mga import MGA

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_analysis(model,depot,nodes,num_vehicle):
    start = time.time()
    model.set_model(nodes = nodes,depot = depot, num_vehicle = num_vehicle)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    results = {"solution":[[i["id"] for i in sol] for sol in solution],
              "fitness":fitness,
              "run_time":run_time}
    
    return results

In [3]:
dir_path = "raw_data/MCTOPMTW-Solomon"
files = glob.glob(os.path.join(dir_path,"*.txt"))

In [4]:
def get_data(filename):
    print(filename)
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    #find the start of the line
    start = 1
    
    parts = list(map(float,lines[start].strip().split()))
    depot = {
        "id": parts[0], #id
        "x": parts[1], #x coordinate
        "y": parts[2], #y coordinate
        "d": parts[3], #service duration or visiting time
        "S": parts[4], #profit of the location
        "O": parts[5], #opening of time window (earliest time for start of service) for depot its a starting point
        "C": parts[6], #closing of time window (latest time for start of service) for depot its the maximum duration (Tmax)
    }

    nodes = []
    
    for line in lines[start+1:]:
        parts = list(map(float, line.strip().split()))
        if len(parts) >= 9:
            node = {
                "id": parts[0], #id
                "x": parts[1], #x coordinate
                "y": parts[2], #y coordinate
                "d": parts[3], #service duration or visiting time
                "S": parts[4], #profit of the location
                "O": parts[5], #opening of time window (earliest time for start of service, use O1)
                "C": parts[9], #closing of time window (latest time for start of service, use C4)
                "b": parts[11], #entrance fee
            }
            nodes.append(node)
        else:
            continue
    return depot,nodes

In [ ]:
df_results = pd.DataFrame()

#scenario
num_vehicle = 3 #ubah nilai ini untuk setiap skenario jumlah vehicle/hari
random_states = list(range(5,(5*20)+1,5))

for file in files:
    depot,nodes = get_data(file)
    for random_state in random_states:

        results = {
            "method" : [],
            "fitness" : [],
            "run_time" : []
        }

#         aco = ACO(random_state = random_state)
#         results_aco = get_analysis(aco,depot,nodes,num_vehicle)
#         results["method"].append("aco")

#         acs = ACS(random_state = random_state)
#         results_acs = get_analysis(acs,depot,nodes,num_vehicle)
#         results["method"].append("acs")

#         bso = BSO(random_state = random_state)
#         results_bso = get_analysis(bso,depot,nodes,num_vehicle)
#         results["method"].append("bso")

        dka = DKA(random_state = random_state)
        results_dka = get_analysis(dka,depot,nodes,num_vehicle)
        results["method"].append("dka")

#         ga = GA(random_state = random_state)
#         results_ga = get_analysis(ga,depot,nodes,num_vehicle)
#         results["method"].append("ga")

#         pso = PSO(random_state = random_state)
#         results_pso = get_analysis(pso,depot,nodes,num_vehicle)
#         results["method"].append("pso")

#         sa = SA(random_state = random_state)
#         results_sa = get_analysis(sa,depot,nodes,num_vehicle)
#         results["method"].append("sa")

#         ts = TS(random_state = random_state)
#         results_ts = get_analysis(ts,depot,nodes,num_vehicle)
#         results["method"].append("ts")

#         woa = WOA(random_state = random_state)
#         results_woa = get_analysis(woa,depot,nodes,num_vehicle)
#         results["method"].append("woa")
        
        ils = ILS(random_state = random_state)
        results_ils = get_analysis(ils,depot,nodes,num_vehicle)
        results["method"].append("ils")
        
#         ils_sa = ILS_SA(random_state = random_state)
#         results_ils_sa = get_analysis(ils_sa,depot,nodes,num_vehicle)
#         results["method"].append("ils_sa")
        
#         ils_ts = ILS_TS(random_state = random_state)
#         results_ils_ts = get_analysis(ils_ts,depot,nodes,num_vehicle)
#         results["method"].append("ils_ts")
        
#         ils_sa_ts = ILS_SA_TS(random_state = random_state)
#         results_ils_sa_ts = get_analysis(ils_sa_ts,depot,nodes,num_vehicle)
#         results["method"].append("ils_sa_ts")
        
#         mga = MGA(random_state = random_state)
#         results_mga = get_analysis(mga,depot,nodes,num_vehicle)
#         results["method"].append("mga")

        for i in [results_aco,results_acs,results_bso,results_dka,results_ga,
                  results_pso,results_sa,results_ts,results_woa,results_ils,
                  results_ils_sa,results_ils_ts,results_ils_sa_ts,results_mga]:
            results["fitness"].append(i["fitness"])
            results["run_time"].append(i["run_time"])

        results = pd.DataFrame(results)
        results["dataset"] = file.split("\\")[-1]
        results["random_state"] = random_state

        df_results = pd.concat([df_results,results])
        print("========================================")

In [6]:
df_results

,method,fitness,run_time,dataset
0,aco,570.0,56.822612,c101.txt
1,acs,320.0,23.056549,c101.txt
2,bso,260.0,6.900478,c101.txt
3,dka,340.0,3.057640,c101.txt
4,ga,360.0,9.767205,c101.txt
...,...,...,...,...
4,ga,374.0,35.867653,rc108.txt
5,pso,365.0,2.382843,rc108.txt
6,sa,485.0,9.011644,rc108.txt
7,ts,418.0,25.180608,rc108.txt


In [8]:
df_results.to_excel(f"dka_public_dataset_mctoptw_{num_vehicle}.xlsx",index=False)

# Uji Statistik

In [3]:
def uji_statistik(first_data,second_data,alternative="two-sided",first_data_name="first",second_data_name="second"):
    # first_data is the target data, e.g. the results of Hybrid ACS-BSO 
    # second_data is a list of results from other method
    # the test is using Wilcoxon Rank-Sum Test
    p_values = []
    second_data_name = ["second" for i in range(len(second_data))] if second_data_name == "second" else second_data_name
    for i in range(len(second_data)):
        statistic, p_value = stats.ranksums(first_data, second_data[i],alternative=alternative)
        p_values.append(p_value)
        print(f"Wilcoxon Rank-Sum Test ({first_data_name} vs. {second_data_name[i]}):")
        print("Test Statistic:", statistic)
        print("p-value:", p_value)
        print("")
    return p_values

def save_to_excel_sheet(df,filename,sheet_name):
    try:
        with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:

            # use to_excel function and specify the sheet_name and index to 
            # store the dataframe in specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    except:
        with pd.ExcelWriter(filename, mode="w", engine="openpyxl") as writer:

            # use to_excel function and specify the sheet_name and index to 
            # store the dataframe in specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [5]:
results_df = pd.read_excel(f"dka_public_dataset_mctoptw_{num_vehicle}.xlsx")
results_df["group"] = results_df["dataset"].str.extract(r"^(rc|c|r)")
results_df.head()

,method,fitness,run_time,dataset,random_state,group
0,aco,580,9.010226,c101.txt,10,c
1,acs,260,3.160357,c101.txt,10,c
2,bso,110,0.829833,c101.txt,10,c
3,dka,390,0.489593,c101.txt,10,c
4,ga,370,1.867269,c101.txt,10,c


In [10]:
grouped = results_df.groupby(['group','dataset','method']).agg(fitness = ('fitness','mean'),
                                                               std_fitness = ('fitness','std'),
                                                               best_fitness = ('fitness','max'),
                                                               run_time = ('run_time','mean'),
                                                               std_run_time = ('run_time','std'),
                                                               best_run_time = ('run_time','min')).reset_index()

In [11]:
grouped

,group,dataset,method,fitness,std_fitness,best_fitness,run_time,std_run_time,best_run_time
0,c,c101.txt,aco,579.0,28.460499,650,9.174792,1.015220,7.926972
1,c,c101.txt,acs,328.0,45.898439,420,4.031451,0.637326,3.160357
2,c,c101.txt,bso,228.0,60.147966,350,0.744833,0.363515,0.201420
3,c,c101.txt,dka,369.0,27.264140,410,0.655700,0.224979,0.441113
4,c,c101.txt,ga,423.0,42.700507,480,3.408050,1.581302,1.468902
...,...,...,...,...,...,...,...,...,...
401,rc,rc108.txt,mga,383.4,53.742596,477,0.129242,0.026859,0.103225
402,rc,rc108.txt,pso,397.4,26.982299,447,0.631933,0.277517,0.361484
403,rc,rc108.txt,sa,497.7,46.166486,562,1.410835,0.074420,1.355216
404,rc,rc108.txt,ts,418.4,5.758086,431,3.451912,0.257101,3.267082


In [12]:
# group_dataset = ["c","r","rc"]
group_dataset = results_df['dataset'].unique()

df_stat = pd.DataFrame()
for group in group_dataset:
    print("group: ",group)
    metrics = "fitness"
    alternative = "two-sided"
    first_data = results_df[(results_df['method']=="dka")&
                            (results_df['dataset']==group)][metrics].values
    first_data_name = "dka"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka"])]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&
                                      (results_df['dataset']==group)][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'dataset':group,
                                               'metrics':metrics,
                                               'method':second_data_name,
                                               'p_value':p_values})])

group:  c101.txt
Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: 2.192193943453518
p-value: 0.028365505605209992

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: 3.5906624935876588
p-value: 0.00032983852077799353

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -2.5701584164627453
p-value: 0.01016520189195626

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -3.7418482827913495
p-value: 0.00018267179110955002

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.4772731516848907
p-value: 0.00050654148469229

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: 2.834733547569204
p-value: 0.004586392080253494

Wilcoxon Rank-Sum Test (dka vs. ils_sa):


Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: 2.721344205666436
p-value: 0.006501702373081825

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: 3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -2.2299903907544407
p-value: 0.025748080821108063

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -1.927618812347059
p-value: 0.053902557169387175

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -0.8693182879212227
p-value: 0.3846730627355087

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: 2.0410081542498273
p-value: 0.04125001659393949

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statistic: -1.17

Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -3.250494467879354
p-value: 0.0011520450981421845

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: 3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.1749015732775088
p-value: 0.001498873337151676

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -3.5906624935876588
p-value: 0.00032983852077799353

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.250494467879354
p-value: 0.0011520450981421845

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: -0.6047431568147635
p-value: 0.5453496680111236

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statistic

Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.704051835490427
p-value: 0.00021218287122257823

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: 0.15118578920369088
p-value: 0.8798291600118298

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: 3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.023715784073818
p-value: 0.002496908915141548

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -2.872529994870127
p-value: 0.004071994217732759

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -1.4740614447359863
p-value: 0.14046504815835495

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -2.4945655218609
p-value: 0.012611144099313947

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: 0.0
p-value: 1.0

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statistic: -2.418972627259054
p-value: 0.0155644113

Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -1.9654152596479817
p-value: 0.0493661947519327

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: 3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -2.418972627259054
p-value: 0.015564411386633814

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -3.6284589408885815
p-value: 0.00028511808363161265

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.288290915180277
p-value: 0.0010079762403767444

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -2.872529994870127
p-value: 0.004071994217732759

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: -1.889822365046136
p-value: 0.05878172135535886

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statistic: -3.

In [13]:
# group_dataset = ["c","r","rc"]
group_dataset = results_df['dataset'].unique()

df_stat2 = pd.DataFrame()
for group in group_dataset:
    print("group: ",group)
    metrics = "run_time"
    alternative = "two-sided"
    first_data = results_df[(results_df['method']=="dka")&
                            (results_df['dataset']==group)][metrics].values
    first_data_name = "dka"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka"])]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&
                                      (results_df['dataset']==group)][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat2 = pd.concat([df_stat2,pd.DataFrame({'dataset':group,
                                               'metrics':metrics,
                                               'method':second_data_name,
                                               'p_value':p_values})])

group:  c101.txt
Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: -0.4535573676110727
p-value: 0.6501474440948545

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: 0.30237157840738177
p-value: 0.7623688184698398

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ils_sa

Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: -1.360672102833218
p-value: 0.17361733442494354

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: 1.1338934190276817
p-value: 0.25683925795785656

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.704051835490427
p-value: 0.00021218287122257823

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: -3.704051835490427
p-value: 0.00021218287122257823

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statistic

Wilcoxon Rank-Sum Test (dka vs. mga):
Test Statistic: 3.779644730092272
p-value: 0.00015705228423075119

group:  r104.txt
Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: -1.5874507866387544
p-value: 0.1124105846553637

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -2.418972627259054
p-value: 0.015564411386633814

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ils):


Wilcoxon Rank-Sum Test (dka vs. aco):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. acs):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. bso):
Test Statistic: -0.15118578920369088
p-value: 0.8798291600118298

Wilcoxon Rank-Sum Test (dka vs. ga):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. pso):
Test Statistic: -0.8315218406202999
p-value: 0.40567889528505297

Wilcoxon Rank-Sum Test (dka vs. sa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ts):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. woa):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ils):
Test Statistic: -3.779644730092272
p-value: 0.00015705228423075119

Wilcoxon Rank-Sum Test (dka vs. ils_sa):
Test Statist

In [14]:
df_stat

,dataset,metrics,method,p_value
0,c101.txt,fitness,aco,0.000157
1,c101.txt,fitness,acs,0.028366
2,c101.txt,fitness,bso,0.000330
3,c101.txt,fitness,ga,0.010165
4,c101.txt,fitness,pso,0.000183
...,...,...,...,...
8,rc108.txt,fitness,ils,0.939743
9,rc108.txt,fitness,ils_sa,0.023342
10,rc108.txt,fitness,ils_ts,0.705457
11,rc108.txt,fitness,ils_sa_ts,0.096304


In [15]:
df_stat2

,dataset,metrics,method,p_value
0,c101.txt,run_time,aco,0.000157
1,c101.txt,run_time,acs,0.000157
2,c101.txt,run_time,bso,0.650147
3,c101.txt,run_time,ga,0.000157
4,c101.txt,run_time,pso,0.762369
...,...,...,...,...
8,rc108.txt,run_time,ils,0.001499
9,rc108.txt,run_time,ils_sa,0.000157
10,rc108.txt,run_time,ils_ts,0.000157
11,rc108.txt,run_time,ils_sa_ts,0.000157


In [18]:
results_df

,method,fitness,run_time,dataset,random_state,group
0,aco,580,9.010226,c101.txt,10,c
1,acs,260,3.160357,c101.txt,10,c
2,bso,110,0.829833,c101.txt,10,c
3,dka,390,0.489593,c101.txt,10,c
4,ga,370,1.867269,c101.txt,10,c
...,...,...,...,...,...,...
4055,ils,327,1.697811,rc108.txt,100,rc
4056,ils_sa,455,7.932625,rc108.txt,100,rc
4057,ils_ts,375,7.238199,rc108.txt,100,rc
4058,ils_sa_ts,348,8.011159,rc108.txt,100,rc


In [23]:
fitness_pivot = grouped.pivot(index=["dataset"],columns="method",values="fitness").reset_index().copy()
run_time_pivot = grouped.pivot(index=["dataset"],columns="method",values="run_time").reset_index().copy()

In [15]:
grouped1 = grouped.copy()
grouped1['metrics'] = 'fitness'
grouped1 = grouped1.pivot(index=['metrics','dataset'],columns='method',values='fitness').reset_index()

grouped2 = grouped.copy()
grouped2['metrics'] = 'run_time'
grouped2 = grouped2.pivot(index=['metrics','dataset'],columns='method',values='run_time').reset_index()

grouped3 = grouped.copy()
grouped3['metrics'] = 'std_fitness'
grouped3 = grouped3.pivot(index=['metrics','dataset'],columns='method',values='std_fitness').reset_index()

grouped4 = grouped.copy()
grouped4['metrics'] = 'std_run_time'
grouped4 = grouped4.pivot(index=['metrics','dataset'],columns='method',values='std_run_time').reset_index()

grouped5 = grouped.copy()
grouped5['metrics'] = 'best_fitness'
grouped5 = grouped5.pivot(index=['metrics','dataset'],columns='method',values='best_fitness').reset_index()

grouped6 = grouped.copy()
grouped6['metrics'] = 'best_run_time'
grouped6 = grouped6.pivot(index=['metrics','dataset'],columns='method',values='best_run_time').reset_index()

grouped = pd.concat([grouped1,grouped2,grouped3,grouped4,grouped5,grouped6])

In [16]:
grouped[grouped['metrics']=="best_fitness"]

method,metrics,dataset,aco,acs,bso,dka,ga,ils,ils_sa,ils_sa_ts,ils_ts,mga,pso,sa,ts,woa
0,best_fitness,c101.txt,650.0,420.0,350.0,410.0,480.0,390.0,520.0,600.0,460.0,550.0,490.0,670.0,540.0,530.0
1,best_fitness,c102.txt,710.0,650.0,370.0,560.0,600.0,540.0,590.0,670.0,550.0,610.0,620.0,780.0,660.0,580.0
2,best_fitness,c103.txt,760.0,720.0,370.0,620.0,620.0,590.0,630.0,670.0,550.0,700.0,660.0,830.0,690.0,600.0
3,best_fitness,c104.txt,800.0,790.0,460.0,610.0,690.0,620.0,640.0,630.0,580.0,680.0,730.0,870.0,730.0,750.0
4,best_fitness,c105.txt,650.0,540.0,360.0,560.0,580.0,450.0,590.0,550.0,500.0,580.0,570.0,730.0,600.0,560.0
5,best_fitness,c106.txt,590.0,490.0,240.0,390.0,460.0,390.0,480.0,510.0,450.0,510.0,500.0,620.0,480.0,430.0
6,best_fitness,c107.txt,760.0,540.0,370.0,560.0,570.0,490.0,620.0,580.0,600.0,600.0,590.0,770.0,690.0,570.0
7,best_fitness,c108.txt,780.0,530.0,390.0,610.0,550.0,530.0,640.0,630.0,560.0,660.0,620.0,810.0,730.0,640.0
8,best_fitness,c109.txt,860.0,670.0,460.0,590.0,650.0,560.0,670.0,670.0,560.0,630.0,720.0,810.0,840.0,650.0
9,best_fitness,r101.txt,313.0,267.0,202.0,262.0,295.0,317.0,303.0,331.0,252.0,326.0,328.0,382.0,274.0,338.0


In [28]:
df_stat = df_stat.pivot(index=['metrics','dataset'],columns='method',values='p_value').reset_index()
df_stat2 = df_stat2.pivot(index=['metrics','dataset'],columns='method',values='p_value').reset_index()
df_stat = pd.concat([df_stat,df_stat2])

In [29]:
df_stat

method,metrics,dataset,aco,acs,bso,ga,ils,ils_sa,ils_sa_ts,ils_ts,mga,pso,sa,ts,woa
0,fitness,c101.txt,0.000157,0.028366,0.000330,0.010165,0.004586,0.001315,0.112411,0.850107,0.000212,0.000183,0.000157,0.000157,0.000507
1,fitness,c102.txt,0.000157,0.596701,0.000183,0.064022,0.075662,0.037635,0.015564,0.325751,0.011330,0.025748,0.000157,0.000157,0.069642
2,fitness,c103.txt,0.000157,0.000285,0.000157,0.088973,0.344704,0.969850,0.023342,0.069642,0.001499,0.001008,0.000157,0.000157,0.082099
3,fitness,c104.txt,0.000157,0.000157,0.000157,0.000583,0.520523,0.130570,0.307489,0.364346,0.000583,0.000769,0.000157,0.000157,0.003611
4,fitness,c105.txt,0.000157,0.028366,0.000212,0.007285,0.104110,0.003611,0.058782,0.064022,0.002827,0.012611,0.000157,0.000440,0.096304
5,fitness,c106.txt,0.000157,0.364346,0.000157,0.012611,0.241322,0.015564,0.001315,0.762369,0.000881,0.000246,0.000157,0.000157,0.002497
6,fitness,c107.txt,0.000157,0.006502,0.000157,0.025748,0.041250,0.241322,0.005159,0.705457,0.001008,0.053903,0.000157,0.000157,0.384673
7,fitness,c108.txt,0.000157,0.161972,0.000157,0.344704,0.307489,0.344704,0.121225,0.405679,0.017257,0.011330,0.000157,0.000157,0.069642
8,fitness,c109.txt,0.000157,0.623176,0.000157,0.000881,0.021134,0.025748,0.003197,0.008151,0.053903,0.000381,0.000157,0.000157,0.007285
9,fitness,r101.txt,0.000157,0.677585,0.000212,0.000507,0.570750,0.004072,0.000769,0.025748,0.001152,0.001499,0.000157,0.185877,0.006502


In [ ]:
save_to_excel_sheet(fitness_pivot,filename="dka_experiment_results_public_mctoptw.xlsx",sheet_name=f"fitness values all vec {num_vehicle}")
save_to_excel_sheet(run_time_pivot,filename="dka_experiment_results_public_mctoptw.xlsx",sheet_name=f"run time all vec {num_vehicle}")
save_to_excel_sheet(grouped,filename="dka_experiment_results_public_mctoptw.xlsx",sheet_name=f"results aggregated vec {num_vehicle}")
save_to_excel_sheet(df_stat,filename="dka_experiment_results_public_mctoptw.xlsx",sheet_name=f"results p_value vec {num_vehicle}")